In [7]:
import csv, os
import matplotlib.pyplot as plt
import numpy as np

def read_csv(dir, run, d=1):
    filename = os.path.join(dir, run, "logs.csv")
    steps, eps_ret, eps_n = [], [], []

    with open(filename, 'r') as file:
        csvreader = csv.reader(file)
        header = next(csvreader)
        for row in csvreader:            
            if len(row) > 0 and int(row[-1]) > 100:
                steps.append(int(row[2])/d)
                eps_ret.append(float(row[-2]))
                #eps_n.append(row[-1])
    return steps, eps_ret

def simplify(steps, eps_ret, gap=100000):
    ind = 0
    new_steps, new_eps_ret = [], []
    for n, (i, j) in enumerate(zip(steps, eps_ret)):
        if i > ind:
            new_steps.append(i)
            new_eps_ret.append(j)
            ind += gap
    return new_steps, new_eps_ret
    
def mv(a, n=1000) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n    

In [122]:
servers = ["uk", 
           "cam", 
           "v1", 
           "v2", 
           "v3", 
           "v4"]
latest_runs = ["lstm_3_1_rec_n_5_gbste_aug", 
               "alstm_3_1_rec_n_5_rei_aug",
               "alstm_3_1_rec_n_5_gbste_aug",
               "alstm_3_1_rec_n_10_rei",
               "alstm_3_1_rec_n_5_rei_aug_2",
               "alstm_3_1_rec_n_10_gbste_aug",]               

for server, lr in zip(servers, latest_runs):
    !mkdir ~/RS/thinker/logs/planner_logs/$lr
    !scp $server:~/RS/thinker/logs/torchbeast/latest/*.csv ~/RS/thinker/logs/planner_logs/$lr

mkdir: cannot create directory ‘/home/sc/RS/thinker/logs/planner_logs/lstm_3_1_rec_n_5_gbste_aug’: File exists
fields.csv                                    100%    0     0.0KB/s   00:00    
logs.csv                                      100%   25MB 106.3MB/s   00:00    
mkdir: cannot create directory ‘/home/sc/RS/thinker/logs/planner_logs/alstm_3_1_rec_n_5_rei_aug’: File exists
fields.csv                                    100%    0     0.0KB/s   00:00    
logs.csv                                      100%   95MB  16.0MB/s   00:05    
mkdir: cannot create directory ‘/home/sc/RS/thinker/logs/planner_logs/alstm_3_1_rec_n_5_gbste_aug’: File exists
Welcome to vast.ai. If authentication fails, try again after a few seconds, and double check your ssh key.
Have fun!
fields.csv                                    100%    0     0.0KB/s   00:00    
logs.csv                                      100%   11MB   1.5MB/s   00:07    
mkdir: cannot create directory ‘/home/sc/RS/thinker/logs/planner_logs/

In [ ]:
#%matplotlib inline
%matplotlib qt

runs = os.listdir('logs/planner_logs')
steps, eps_rets = {}, {}
mv_avg = 20

for run in runs:
    if "." in run or "past" == run: continue
    if "rei" in run:
        st = run.find("rec_n_")
        d = int(run[st+6:run.find("_", st+6)])
    else:
        d = 1  
    step, eps_ret = simplify(*read_csv('logs/planner_logs', run, d))
    bound = np.nonzero(np.array(step) > int(1e8))[0]
    if len(bound) >= 1:
        step = step[:bound[0]]
        eps_ret = eps_ret[:bound[0]]
    steps[run] = step
    eps_rets[run] = eps_ret

In [132]:
f, ax = plt.subplots(1, 1, figsize=(12,7))

linestyles=["solid", "dashed", "dotted","dashdot"]
n = 0
for _, run in enumerate(eps_rets.keys()): 
    if "gbste" not in run: continue    
    ax.plot(np.array(steps[run][mv_avg-1:]), 
            mv(eps_rets[run], n=mv_avg), 
            label=run, 
            linestyle=linestyles[n//10],
            linewidth=1)
    n += 1
plt.xlabel('Step')
plt.ylabel('Eps return')
plt.axhline(y=0.12, color='r', label="base policy", linewidth=1)
plt.axhline(y=1.08, color='g', label="one-step greedy search", linewidth=1)
plt.axhline(y=1.17, color='y', label="two-step greedy search", linewidth=1)
plt.tight_layout() 
plt.legend()
plt.show()    